# Implémentation basique du simplexe

In [1]:
## https://github.com/chkwon/jpor_codes/blob/master/chap5/simplex_method.jl
## Slightly modified to avoid the inversion operation

using LinearAlgebra, Combinatorics, Printf

mutable struct SimplexTableau
    z_c     ::Array{Float64} # z_j - c_j
    Y       ::Array{Float64} # inv(B) * A
    x_B     ::Array{Float64} # inv(B) * b
    obj     ::Float64        # c_B * x_B
    b_idx   ::Array{Int64}   # indices for basic variables x_B
end
function is_nonnegative(x::Vector)
    return length( x[ x .< 0] ) == 0
end

function is_nonpositive(z::Array)
    return length( z[ z .> 0] ) == 0
end

function initial_BFS(A, b)
    m, n = size(A)

    comb = collect(combinations(1:n, m))
    for i in length(comb):-1:1
      b_idx = comb[i]
      B = A[:, b_idx]
      if rank(B) == m        
        x_B = B\b
        if is_nonnegative(x_B)
          return b_idx, x_B, B
        end
      end
    end

    error("Infeasible")
end

function print_tableau(t::SimplexTableau)
    m, n = size(t.Y)

    hline0 = repeat("-", 6)
    hline1 = repeat("-", 7*n)
    hline2 = repeat("-", 7)
    hline = join([hline0, "+", hline1, "+", hline2])

    println(hline)

    @printf("%6s|", "")
    for j in 1:length(t.z_c)
      @printf("%6.2f ", t.z_c[j])
    end
    @printf("| %6.2f\n", t.obj)

    println(hline)

    for i in 1:m
      @printf("x[%2d] |", t.b_idx[i])
      for j in 1:n
        @printf("%6.2f ", t.Y[i,j])
      end
      @printf("| %6.2f\n", t.x_B[i])
    end

    println(hline)
end

function pivoting!(t::SimplexTableau)
    m, n = size(t.Y)

    entering, exiting = pivot_point(t)
    println("Pivoting: entering = x_$entering, exiting = x_$(t.b_idx[exiting])")

    # Pivoting: exiting-row, entering-column
    # updating exiting-row
    coef = t.Y[exiting, entering]
    t.Y[exiting, :] /= coef
    t.x_B[exiting] /= coef

    # updating other rows of Y
    for i in setdiff(1:m, exiting)
      coef = t.Y[i, entering]
      t.Y[i, :] -= coef * t.Y[exiting, :]
      t.x_B[i] -= coef * t.x_B[exiting]
    end

    # updating the row for the reduced costs
    coef = t.z_c[entering]
    t.z_c -= coef * t.Y[exiting, :]'
    t.obj -= coef * t.x_B[exiting]

    # Updating b_idx
    t.b_idx[ findfirst(t.b_idx .== t.b_idx[exiting]) ] = entering
end

function pivot_point(t::SimplexTableau)
    # Finding the entering variable index
    entering = findfirst( t.z_c .> 0)[2]
    if entering == 0
      error("Optimal")
    end

    # min ratio test / finding the exiting variable index
    pos_idx = findall( t.Y[:, entering] .> 0 )
    if length(pos_idx) == 0
      error("Unbounded")
    end
    exiting = pos_idx[ argmin( t.x_B[pos_idx] ./ t.Y[pos_idx, entering] ) ]

    return entering, exiting
end

function initialize(c, A, b)
    c = Array{Float64}(c)
    A = Array{Float64}(A)
    b = Array{Float64}(b)

    m, n = size(A)

    # Finding an initial BFS
    b_idx, x_B, B = initial_BFS(A,b)

    Y = B\A
    c_B = c[b_idx]
    obj = dot(c_B, x_B)

    # z_c is a row vector
    z_c = zeros(1,n)
    n_idx = setdiff(1:n, b_idx)
    λ = B\c_B
    z_c[n_idx] = λ' * A[:,n_idx] - c[n_idx]'

    return SimplexTableau(z_c, Y, x_B, obj, b_idx)
end

function is_optimal(t::SimplexTableau)
    return is_nonpositive(t.z_c)
end

function simplex_method(c, A, b)
    tableau = initialize(c, A, b)
    #print_tableau(tableau)

    while !is_optimal(tableau)
      pivoting!(tableau)
      #print_tableau(tableau)
    end

    opt_x = zeros(length(c))
    opt_x[tableau.b_idx] = tableau.x_B

    return opt_x, tableau.obj
end

simplex_method (generic function with 1 method)

## Phase 1

Notre solution utilisera la phase 1 pour créer un tableau initiale du simplexe. Ceci ce fera en 3 étapes

    1 créer le problème de phase 1
    2 résoudre le problème de phase 1
    3 aller chercher l'information dans le tableau optimale du simplexe pour créer le tableau du simplexe du problème original

In [2]:
function phase1(c,A,b)
    
    # Étape 1, créer le problème de phase 1
    m,n = size(A)
    c = Array{Float64}(c)
    A = Array{Float64}(A)
    b = Array{Float64}(b)
    #on ajoute de façon naive une variable artificiel pour chaque contrainte
    Aphase1 = [A I]
    # on doit minimiser la somme des variables artificiel
    cphase1 = [zeros(n)' ones(m)']
    # on initialise la valeur l'objectif à 0.0
    objphase1 = 0.0
    #on sait que la base initiale contient les  variables artificiels [n+1, n+2,..., n+m]
    b_idxphase1 = [n+1:n+m;]
    # la solution de base, composée des variables artificielles, vaut initialement le terme de droite
    x_Bphase1 = copy(b)
    
    # Dans la forme standard, le vecteur c devrait contenir un 0 sous les variable de base
    # On sait que dans la forme actuel, le couts réduits sous les variable de base sont 1.
    # On met donc les coûts réduits de base à 0 en soustrayant chaque ligne de la matrice.
    for k in 1:m
        cphase1[1,:] -= Aphase1[k, :]
        objphase1 -= b[k]
    end
    
    # On a maintenant tous ce qu'il faut pour créer le tableau du  simplexe!!!
    stphase1 = SimplexTableau(-cphase1, Aphase1, x_Bphase1, -objphase1, b_idxphase1)
    print_tableau(stphase1)
       
    # Partie 2: résoudre le problème de phase 1
    while !is_optimal(stphase1)
      pivoting!(stphase1)
      print_tableau(stphase1)
    end
    
    # On a maintenant un tableau optimal, il ne reste qu'à faire quelque vérification
    # le problème est-il réalisable ?
    @assert stphase1.obj <= eps() "infeasible, obj phase1 = $(stphase1.obj)"
    
    # Partie 3: aller chercher l'information dans le tableau du simplexe de phase 1.
    cphase2 = copy(c)
    objphase2 = 0.0
    
    # On va retirer les contraintes redondantes, ceci ne sera pas évalué dans la correction.
    Y = zeros(0, n)
    b_phase2 = zeros(0)
    b_idx = Int[]
    # Pour toutes les contraintes
    for (k, bk) in enumerate(stphase1.b_idx)
        #si la variable de base associé à la contrainte n'est pas une variable de phase 1
        if bk != n+1:n+m
            push!(b_idx, bk)
            Y = [Y; stphase1.Y[k, 1:n]']
            push!(b_phase2, stphase1.x_B[k])
            objphase2 -= cphase2[1,bk]*b_phase2[end]
            cphase2[1,:] -= cphase2[1,bk]*Y[end, 1:n]
        #si la variable de base associé à la contrainte est pas une variable de phase 1
        else
            #on n'ajoute pas la contrainte et on retire indique qu'on a retirer une ligne
            println("line $k removed")
        end
    end

    # Il ne reste qu'à retourner le résultat!!
    tableau = SimplexTableau(-cphase2, Y, b_phase2, -objphase2, b_idx)
    print_tableau(tableau)

    return tableau
end

phase1 (generic function with 1 method)

In [3]:
function simplex_method2(c, A, b)
    tableau = phase1(c, A, b)
    print_tableau(tableau)

    while !is_optimal(tableau)
      pivoting!(tableau)
      print_tableau(tableau)
    end
    print_tableau(tableau)
    opt_x = zeros(length(c))
    opt_x[tableau.b_idx] = tableau.x_B

    return opt_x, tableau.obj
end

simplex_method2 (generic function with 1 method)

## Exemple

In [4]:
c = reshape([-3; -2; -1; -5; 0; 0; 0], 1, 7)
A = [7 3 4 1 1 0 0 ;
     2 1 1 5 0 1 0 ;
     1 4 5 2 0 0 1 ]
b = [7; 3; 8]

x, obj = simplex_method(c, A, b)
obj

Pivoting: entering = x_1, exiting = x_5
Pivoting: entering = x_2, exiting = x_7
Pivoting: entering = x_4, exiting = x_6


-5.051724137931035

In [5]:
c = reshape([-3; -2; -1; -5; 0; 0; 0], 1, 7)
A = [7 3 4 1 1 0 0 ;
     2 1 1 5 0 1 0 ;
     1 4 5 2 0 0 1 ]
b = [7; 3; 8]

x, obj = simplex_method2(c, A, b)
obj

------+----------------------------------------------------------------------+-------
      | 10.00   8.00  10.00   8.00   1.00   1.00   1.00  -0.00  -0.00  -0.00 |  18.00
------+----------------------------------------------------------------------+-------
x[ 8] |  7.00   3.00   4.00   1.00   1.00   0.00   0.00   1.00   0.00   0.00 |   7.00
x[ 9] |  2.00   1.00   1.00   5.00   0.00   1.00   0.00   0.00   1.00   0.00 |   3.00
x[10] |  1.00   4.00   5.00   2.00   0.00   0.00   1.00   0.00   0.00   1.00 |   8.00
------+----------------------------------------------------------------------+-------
Pivoting: entering = x_1, exiting = x_8
------+----------------------------------------------------------------------+-------
      |  0.00   3.71   4.29   6.57  -0.43   1.00   1.00  -1.43  -0.00  -0.00 |   8.00
------+----------------------------------------------------------------------+-------
x[ 1] |  1.00   0.43   0.57   0.14   0.14   0.00   0.00   0.14   0.00   0.00 |   1.00
x[ 9] |  0.00 

-5.051724137931034

In [6]:
x

7-element Array{Float64,1}:
 0.17241379310344832
 1.8793103448275859
 0.0
 0.15517241379310345
 0.0
 0.0
 0.0

# Résoudre le problème en question 1

In [7]:
A = [[1 0 1 0; 0 1  2 1; 0 0 0 1] -I]
c = reshape([[0,3,4,5] ; zeros(3)], 1, 7)
b = [3,4,2]

3-element Array{Int64,1}:
 3
 4
 2

In [8]:
x, obj = simplex_method2(c, A, b)

------+----------------------------------------------------------------------+-------
      |  1.00   1.00   3.00   2.00  -1.00  -1.00  -1.00  -0.00  -0.00  -0.00 |   9.00
------+----------------------------------------------------------------------+-------
x[ 8] |  1.00   0.00   1.00   0.00  -1.00   0.00   0.00   1.00   0.00   0.00 |   3.00
x[ 9] |  0.00   1.00   2.00   1.00   0.00  -1.00   0.00   0.00   1.00   0.00 |   4.00
x[10] |  0.00   0.00   0.00   1.00   0.00   0.00  -1.00   0.00   0.00   1.00 |   2.00
------+----------------------------------------------------------------------+-------
Pivoting: entering = x_1, exiting = x_8
------+----------------------------------------------------------------------+-------
      |  0.00   1.00   2.00   2.00   0.00  -1.00  -1.00  -1.00  -0.00  -0.00 |   6.00
------+----------------------------------------------------------------------+-------
x[ 1] |  1.00   0.00   1.00   0.00  -1.00   0.00   0.00   1.00   0.00   0.00 |   3.00
x[ 9] |  0.00 

([2.0, 0.0, 1.0, 2.0, 0.0, 0.0, 0.0], 14.0)

## Seconde version

Une version plus compacte suit. Pour ce faire, nous modifions légèrement le code du simplexe en permettant de passer directement les indices de la base.

In [9]:
function initialize(c, A, b, b_idx)
    c = Array{Float64}(c)
    A = Array{Float64}(A)
    b = Array{Float64}(b)

    m, n = size(A)

    # We do not call initial_BFS anymore
    B = A[:, b_idx]
    x_B = B\b

    Y = B\A
    c_B = c[b_idx]
    obj = dot(c_B, x_B)

    # z_c is a row vector
    z_c = zeros(1,n)
    n_idx = setdiff(1:n, b_idx)
    λ = B'\c_B
    z_c[n_idx] = λ' * A[:,n_idx] - c[n_idx]'
    
    return SimplexTableau(z_c, Y, x_B, obj, b_idx)
end

function simplex_method(c, A, b, b_idx = [])
    # b_idx is either empty if no basis is known, or contain the indexes of the columns forming the basis
    if b_idx == []
        # Finding an initial BFS
        b_idx, x_B, B = initial_BFS(A,b)
        println(b_idx)
    end
    tableau = initialize(c, A, b, b_idx)
    # print_tableau(tableau)

    while !is_optimal(tableau)
      pivoting!(tableau)
      # print_tableau(tableau)
    end

    opt_x = zeros(length(c))
    opt_x[tableau.b_idx] = tableau.x_B
    
    return opt_x, tableau.obj, tableau.b_idx
  end


simplex_method (generic function with 2 methods)

Les codes de phase 1 et de la méthodes à deux phases suivent.

In [10]:
# Phase I

function phaseI(c, A, b)
    m, n = size(A)
    b_idx = [ i for i = n+1:n+m]
    x, obj, b_idx = simplex_method([zeros(1,n) ones(1,m)], [A I], b, b_idx)
    feasible = true
    
    tol = 1e-8
    if (obj > tol)
        println("Problème non réalisable")
        feasible = false
    end
    
    return feasible, b_idx
end

phaseI (generic function with 1 method)

In [11]:
function simplex_twophases(c, A, b)
    
    x = []; obj = Inf
    feasible, b_idx = phaseI(c, A, b)
    
    if (feasible)
        println("Problème réalisable")
        x, obj, b_idx = simplex_method(c, A, b, b_idx)
    end
    
    return x, obj, b_idx

end    

simplex_twophases (generic function with 1 method)

In [12]:
x, obj, b_idx = simplex_twophases(c, A, b)

Pivoting: entering = x_1, exiting = x_8
Pivoting: entering = x_2, exiting = x_9
Pivoting: entering = x_4, exiting = x_10
Problème réalisable
Pivoting: entering = x_3, exiting = x_2


([2.0, 0.0, 1.0, 2.0, 0.0, 0.0, 0.0], 14.0, [1, 3, 4])